In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, AveragePooling2D, Add
from keras.models import load_model, Model

In [3]:
class CNN:
  # Constructor
  def _init_(self):
    self.model = None

  # Define structure of the CNN
  def build(self, input_dim):
    input = Input(shape=input_dim)

    C1 = Conv2D(6, (3, 3), padding='same', activation='sigmoid')(input)
    S2 = MaxPooling2D(pool_size=(2, 2), padding='same')(C1)
    C3 = Conv2D(16, (3, 3), padding='same', activation='sigmoid')(S2)
    S4 = MaxPooling2D(pool_size=(2, 2), padding='same')(C3)

    flat = Flatten()(S4)
    F5 = Dense(120, activation='sigmoid', use_bias=True)(flat)
    F6 = Dense(84, activation='sigmoid', use_bias=True)(F5)
    
    output = Dense(10, activation='softmax', use_bias=True)(F6)

    self.model = Model(input, output)
    self.model.compile(optimizer="adam", loss = 'categorical_crossentropy')

  # Train the model
  def train(self, x_train, y_train, x_val, y_val):
    self.model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 50, batch_size = 128)

  # Load model from file
  def load(self, model_file):
    self.model = load_model(model_file)

  # Save the trained model
  def save(self, model_file):
    self.model.save(model_file)

  # Show the architecture og the model
  def summary(self):
    self.model.summary()

  # Test the model with a given input
  def predict(self, x_test):
    return self.model.predict(x_test)

In [4]:
class DeltaCNN:
  # Constructor
  def _init_(self):
    self.model = None

  # Define structure of the CNN
  def build(self, input_dim):
    input = Input(shape=input_dim)

    C1 = Conv2D(8, (3, 3), padding='same', activation=None)(input)
    S2 = MaxPooling2D(pool_size=(2, 2), strides = (2, 2), padding='same')(C1)
    C3 = Conv2D(8, (3, 3), padding='same', activation='relu')(S2)
    C4 = Conv2D(8, (3, 3), padding='same', activation='relu')(C3)

    add1 = Add()([S2, C4])
    
    C5 = Conv2D(16, (3, 3), padding='same', activation='sigmoid', strides = (2, 2))(add1)
    C6 = Conv2D(16, (3, 3), padding='same', activation='relu')(C5)
    C7 = Conv2D(16, (3, 3), padding='same', activation='relu')(C6)

    add2 = Add()([C5, C7])

    S8 = AveragePooling2D(pool_size=(2, 2), padding='same')(add2)

    flat = Flatten()(S8)
    F9 = Dense(400, activation='sigmoid', use_bias=True)(flat)

    output = Dense(10, activation='softmax', use_bias=True)(F9)

    self.model = Model(input, output)
    self.model.compile(optimizer="adam", loss = 'categorical_crossentropy')

  # Train the model
  def train(self, x_train, y_train, x_val, y_val):
    self.model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 50, batch_size = 128)

  # Load model from file
  def load(self, model_file):
    self.model = load_model(model_file)

  # Save the trained model
  def save(self, model_file):
    self.model.save(model_file)

  # Show the architecture og the model
  def summary(self):
    self.model.summary()

  # Test the model with a given input
  def predict(self, x_test):
    return self.model.predict(x_test)

In [5]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [6]:
import numpy as np

def onehot(y):
  oh = np.zeros((y.shape[0], 10))
  for i in range(y.shape[0]):
    oh[i, int(y[i])] = 1
  
  return oh

In [7]:
from sklearn import preprocessing

y_train_oh = onehot(y_train)
y_train_oh.shape

y_test_oh = onehot(y_test)
y_test_oh.shape

x_train_norm = x_train / 255
x_test_norm = x_test / 255

x_train_norm = x_train_norm[:, :, :, np.newaxis]
x_test_norm = x_test_norm[:, :, :, np.newaxis]

In [8]:
cnm = DeltaCNN()
cnm.build((28, 28, 1))
cnm.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 28, 28, 8)    80          ['input_1[0][0]']                
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 14, 14, 8)    0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 14, 14, 8)    584         ['max_pooling2d[0][0]']          
                                                                                              

In [9]:
cnm.train(x_train, y_train_oh, x_test, y_test_oh)

Epoch 1/50
469/469 [==============================] - 35s 73ms/step - loss: 0.4574 - val_loss: 0.1411
Epoch 2/50
469/469 [==============================] - 34s 73ms/step - loss: 0.1392 - val_loss: 0.1186
Epoch 3/50
469/469 [==============================] - 35s 74ms/step - loss: 0.1050 - val_loss: 0.0904
Epoch 4/50
469/469 [==============================] - 35s 75ms/step - loss: 0.0857 - val_loss: 0.0843
Epoch 5/50
469/469 [==============================] - 37s 78ms/step - loss: 0.0725 - val_loss: 0.0678
Epoch 6/50
469/469 [==============================] - 35s 75ms/step - loss: 0.0687 - val_loss: 0.0684
Epoch 7/50
469/469 [==============================] - 35s 74ms/step - loss: 0.0638 - val_loss: 0.0587
Epoch 8/50
469/469 [==============================] - 35s 74ms/step - loss: 0.0566 - val_loss: 0.0644
Epoch 9/50
469/469 [==============================] - 35s 74ms/step - loss: 0.0487 - val_loss: 0.0491
Epoch 10/50
469/469 [==============================] - 35s 74ms/step - loss: 0.045